# Obtain SNP Data

## Download and install bgenix
### see https://enkre.net/cgi-bin/code/bgen/doc/trunk/doc/wiki/bgenix.md

In [1]:
cd /opt/notebooks
wget http://code.enkre.net/bgen/tarball/release/bgen.tgz
tar xvfz bgen.tgz > /dev/null
cd bgen.tgz/
./waf configure 
./waf 
./build/test/unit/test_bgen
./build/apps/bgenix -g example/example.16bits.bgen –list
cd /opt/notebooks

--2022-09-27 07:26:07--  http://code.enkre.net/bgen/tarball/release/bgen.tgz
Resolving code.enkre.net (code.enkre.net)... 91.197.228.37
Connecting to code.enkre.net (code.enkre.net)|91.197.228.37|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://enkre.net/cgi-bin/code/bgen/tarball/release/bgen.tgz [following]
--2022-09-27 07:26:07--  https://enkre.net/cgi-bin/code/bgen/tarball/release/bgen.tgz
Resolving enkre.net (enkre.net)... 91.197.228.37
Connecting to enkre.net (enkre.net)|91.197.228.37|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25476745 (24M) [application/x-compressed]
Saving to: ‘bgen.tgz’

bgen.tgz            100%[===================>]  24.30M  43.5MB/s    in 0.6s    

2022-09-27 07:26:25 (43.5 MB/s) - ‘bgen.tgz’ saved [25476745/25476745]

Setting top to                           : /opt/notebooks/bgen.tgz 
Setting out to                           : /opt/notebooks/bgen.tgz/build 
Checking for 'gcc' (C compiler)   

## download and install PLINK2
### see https://www.cog-genomics.org/plink/2.0/
### WARNING: the download link may have to be updated

In [2]:

cd /opt/notebooks
wget https://s3.amazonaws.com/plink2-assets/alpha3/plink2_linux_avx2_20220814.zip
unzip -o plink2_linux_avx2_20220814.zip
./plink2 --version

--2022-09-27 07:27:02--  https://s3.amazonaws.com/plink2-assets/alpha3/plink2_linux_avx2_20220814.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.170.160
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.170.160|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9134949 (8.7M) [application/zip]
Saving to: ‘plink2_linux_avx2_20220814.zip’

plink2_linux_avx2_2 100%[===================>]   8.71M  14.1MB/s    in 0.6s    

2022-09-27 07:27:03 (14.1 MB/s) - ‘plink2_linux_avx2_20220814.zip’ saved [9134949/9134949]

Archive:  plink2_linux_avx2_20220814.zip
  inflating: plink2                  
PLINK v2.00a3.6LM AVX2 Intel (14 Aug 2022)


## Create shortcuts to genotype directories (i.e. to avoid problems with Jupyter using blanks in file names)

In [3]:
DIR=/mnt/project/Bulk/Imputation/UKB*imputation*from*genotype/
ln -sf $DIR /opt/notebooks/impu

DIR=/mnt/project/Bulk/Genotype*Results/Genotype*calls/
ln -sf $DIR /opt/notebooks/geno

# define a function to extract genotyped SNPs [not really needed, just to keep the syntax, we use imputed SNPs below]
function extract_SNP () {
  SNP=${1-rs174547}
  CHR=${2-11}
  echo "extracting SNP $SNP on chromosome $CHR"
  ./plink2 --bfile  geno/ukb22418_c${CHR}_b0_v2 --snp $SNP --export  A  'include-alt' --out ${SNP}
  # head -5 ${SNP}.raw
}
## test the function on a single SNP
extract_SNP rs780094 2


extracting SNP rs780094 on chromosome 2
PLINK v2.00a3.6LM AVX2 Intel (14 Aug 2022)     www.cog-genomics.org/plink/2.0/
(C) 2005-2022 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to rs780094.log.
Options in effect:
  --bfile geno/ukb22418_c2_b0_v2
  --export A include-alt
  --out rs780094
  --snp rs780094

Start time: Tue Sep 27 07:27:06 2022
7605 MiB RAM detected; reserving 3802 MiB for main workspace.
Using up to 4 compute threads.
488377 samples (264746 females, 223430 males, 201 ambiguous; 488377 founders)
loaded from geno/ukb22418_c2_b0_v2.fam.
61966 variants loaded from geno/ukb22418_c2_b0_v2.bim.
1 categorical phenotype loaded (488377 values).
--snp: 1 variant remaining.
1 variant remaining after main filters.
--export A pass 1/1: loading... 0writing... 1010111112121313141415151616171718181919202021212222232324242525262627272828292930303131323233333434353536363737383839394040414142424343444445454646474748484949505051515252535354545555565657575858595960

# Create a list of SNPs to retrieve 

SNPs need to be in the format CHR:POS1-POS2
where individual SNPs are indicated by identical from - to positions
(note that single digit chromosomes need to be indicated using a leading zero)

In [4]:
# crreate a list of SNPs to retrieve 
cat > mypos.txt <<EOF
01:914852-914852
01:2147162-2147162
01:2326009-2326009
22:50746706-50746706
22:50840573-50840573
22:50858813-50858813
EOF

# extract SNPs by position

In [5]:
# genotype data is stored by chromosome, so we loop over these files
CFROM=1
CTO=22

export CHR=$CFROM
while [ $CHR -le $CTO ] ; do

  echo "working on CHR $CHR"
  
  # extract from one BGEN file
  /opt/notebooks/bgen.tgz/build/apps/bgenix -g impu/ukb22828_c${CHR}_b0_v3.bgen -incl-range mypos.txt > mypos.${CHR}.bgen

  # convert to text using PLINK
  ./plink2 --bgen mypos.${CHR}.bgen ref-first --sample impu/ukb22828_c${CHR}_b0_v3.sample --export  A  'include-alt' --out mypos.${CHR} --make-bed
  # head -5 mypos.${CHR}.raw
  wc -l mypos.${CHR}.raw

  ls -ltr mypos.${CHR}*

  ((CHR=CHR+1))
  
done

working on CHR 1

Welcome to bgenix
(version: 1.1.7, revision )

(C) 2009-2017 University of Oxford

Building query                                              :  (3/?,0.6s,4.8/s)
Processing 3 variants                                       : [******************************] (3/3,0.7s,4.2/s)
bgenix: wrote data for 3 variants to stdout.

Thank you for using bgenix.
PLINK v2.00a3.6LM AVX2 Intel (14 Aug 2022)     www.cog-genomics.org/plink/2.0/
(C) 2005-2022 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to mypos.1.log.
Options in effect:
  --bgen mypos.1.bgen ref-first
  --export A include-alt
  --make-bed
  --out mypos.1
  --sample impu/ukb22828_c1_b0_v3.sample

Start time: Tue Sep 27 07:27:19 2022
7605 MiB RAM detected; reserving 3802 MiB for main workspace.
Using up to 4 compute threads.
--bgen: 3 variants detected, format v1.2.
487409 samples imported from .sample file to mypos.1-temporary.psam .
--bgen: mypos.1-temporary.pgen + mypos.1-temporary.pvar writte

# upload to project directory

In [6]:
tar cvfz SNPdata.tgz mypos.*

mypos.1.bed
mypos.1.bgen
mypos.1.bim
mypos.1.fam
mypos.1.log
mypos.1.raw
mypos.10.bgen
mypos.10.log
mypos.11.bgen
mypos.11.log
mypos.12.bgen
mypos.12.log
mypos.13.bgen
mypos.13.log
mypos.14.bgen
mypos.14.log
mypos.15.bgen
mypos.15.log
mypos.16.bgen
mypos.16.log
mypos.17.bgen
mypos.17.log
mypos.18.bgen
mypos.18.log
mypos.19.bgen
mypos.19.log
mypos.2.bgen
mypos.2.log
mypos.20.bgen
mypos.20.log
mypos.21.bgen
mypos.21.log
mypos.22.bed
mypos.22.bgen
mypos.22.bim
mypos.22.fam
mypos.22.log
mypos.22.raw
mypos.3.bgen
mypos.3.log
mypos.4.bgen
mypos.4.log
mypos.5.bgen
mypos.5.log
mypos.6.bgen
mypos.6.log
mypos.7.bgen
mypos.7.log
mypos.8.bgen
mypos.8.log
mypos.9.bgen
mypos.9.log
mypos.txt


In [7]:
dx upload SNPdata.tgz

[===========================================================>] Uploaded 17,487,601 of 17,487,601 bytes (100%) SNPdata.tgz
ID                    file-GGkBGQ0J13J7PKvkPV2x8ZgZ
Class                 file
Project               project-G6v874QJ13J49Z2B0gz538G2
Folder                /
Name                  SNPdata.tgz
State                 closing
Visibility            visible
Types                 -
Properties            -
Tags                  -
Outgoing links        -
Created               Tue Sep 27 07:29:04 2022
Created by            ksuhre
 via the job          job-GGkB2y0J13J6432g9869XFjb
Last modified         Tue Sep 27 07:29:06 2022
Media type            
archivalState         "live"
cloudAccount          "cloudaccount-dnanexus"
